# Set up

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
%matplotlib inline
import networkx as nx

In [ ]:
from Bio import Entrez
from Bio import SeqIO
Entrez.api_key = ""
Entrez.email = ""

In [ ]:
import numpy as np
import pandas as pd

# sparse complete plamisd sequence

## genbank file to annotation(per sequence)

In [ ]:
for Seq_Record in SeqIO.parse("../../biopython/ndm_all_gb.gb","genbank"):
    id=Seq_Record.name
    for feature in Seq_Record.features:
        if feature.type == "source":
            host=feature.qualifiers.get("host")
            country=feature.qualifiers.get("country")
            date=feature.qualifiers.get("collection_date")
            plasmid=feature.qualifiers.get("plasmid")
            organism=feature.qualifiers.get("organism")
    try:
        with open ('./complete/annotation/'+str(organism)+'.'+str(host)+'.'+str(country)+'.'+str(date)+'.'+str(plasmid)+'.'+str(id)+'.tsv','w') as handle:
            handle.write("location\tproduct\n")
        with open ('./complete/annotation/'+str(organism)+'.'+str(host)+'.'+str(country)+'.'+str(date)+'.'+str(plasmid)+'.'+str(id)+'.tsv','a') as handle:
            for feature in Seq_Record.features:
                if feature.type == "CDS":  
            #if feature.qualifiers.get("product") == ['blaNDM-5']:
            #handle.write("{}\t{}\t{}\n".format(Seq_Record.id,feature.qualifiers.get("protein_id"),feature.qualifiers.get("translation")))
                    handle.write("{}\t{}\n".format(feature.location,feature.qualifiers.get("product"))) 
    except Exception as e:
        pass

## give group

In [ ]:
%%bash
cd ./complete/annotation/;
for n in {A..Z}
do 
mkdir ../groupby/${n}/
cp ./[\'${n}* ../groupby/${n}/
done

In [ ]:
%%bash
cd ./complete/groupby/;
find -type d -empty | xargs rm -rf

In [ ]:
%%bash
cd ./complete/groupby/;
num=1;
dir=$(ls)
for n in $dir
do 
ls ${n} >> ${num}.txt
sed -i 's/.tsv//g' ${num}.txt
num=$(($num+1))
done

## output group csv

In [ ]:
import numpy as np
import pandas as pd
group_name=1

while (group_name < 11):
    with open('./complete/groupby/'+str(group_name)+'.txt','r') as f:
        lines = f.read().splitlines()
        for line in lines:
            tsv=pd.read_table('./complete/annotation/'+str(line)+'.tsv')
            #print(tsv[tsv['product']=='Metallo-beta-lactamase type 2'])
            ndm_index=tsv[tsv["product"].str.contains('NDM|ndm|metallo-beta-lactamase|Metallo-beta-lactamase')].index.tolist()
            ndm_pi=[ndm - 20 if ndm >20 else 0 for ndm in ndm_index]
            ndm_ai=[ndm + 21 if ndm+21 < tsv.shape[0] else tsv.shape[0] for ndm in ndm_index]
            for pi,ai in zip(ndm_pi,ndm_ai):
                df=tsv.iloc[pi:ai]
                df.reset_index(drop=True,inplace=True)
                #dfdf[df['product']=='Metallo-beta-lactamase type 2'].index.array
                ndm_df_index=df[df["product"].str.contains('NDM|ndm|metallo-beta-lactamase|Metallo-beta-lactamase')].index.array[0]
                if "-" in df.iloc[ndm_df_index,0]:
                    df=df.iloc[::-1]
                df=df["product"].to_frame()
                #print(df)
                df.reset_index(drop=True,inplace=True)
                ndm_df_index=df[df["product"].str.contains('NDM|ndm|metallo-beta-lactamase|Metallo-beta-lactamase')].index.array[0]
                add_row=20-ndm_df_index
                for i in range(add_row):
                    df.loc[str(i)]='0'
                df=df.shift(periods=add_row,axis=0,fill_value=0)
                df.reset_index(drop=True,inplace=True)
                df=df.T
                df.rename(index={'product':str(line)},inplace=True)
                df.to_csv('./complete/csvgroup/20/'+str(group_name)+'.csv', mode='a', header=False)
    group_name+=1

# spare assembly plasmid sequence

## network file to clique group

In [ ]:
gml_graph = nx.read_gml('./test file/m4all_ndm2.gml')

In [ ]:
ndm_clique=list(nx.find_cliques(gml_graph))

In [ ]:
ndm_include_clique = [clique for clique in ndm_clique if 'blaNDM-1' in clique]

In [ ]:
with open('./assembly/ndm_include_clique'+'.txt','w') as f:
        f.write("{}\n".format("\n".join(str(i) for i in ndm_include_clique)))

## sequence per group

In [ ]:
import pandas as pd
gene_pa=pd.read_table('../output/roary_ndm_assembly_Ecoli/prokka/roary_output2/gene_presence_absence.Rtab')

In [ ]:
group_name=0
for item in ndm1_include_clique:
    I=[]
    df=gene_pa[gene_pa.index.isin(item)]
    df.drop(['Gene'],axis=1,inplace=True)
    for index, col in df.iteritems():
        if col.sum() == df.shape[0]:
            I.append(index)
    #print("{}\n".format(I))
    group_name+=1
    with open('./assembly/ndm_group/'+str(group_name)+'.txt','w') as f:
        f.write("{}\n".format("\n".join(str(i) for i in I)))

## output group csv

In [ ]:
%%bash
cd /mnt/d/biotools/workshop/ndm_plasmid/output/roary_ndm_assembly_Ecoli/prokka_chro/gff;
find -name "*.gff" | xargs grep "NDM" | cut -f 1,7 > gff.txt;
sed -i "s/.*://g" gff.txt;
echo -e "name\tstrand" > gff_uniq.txt;
sort -n gff.txt | uniq >> gff_uniq.txt

In [ ]:
#根据txt分组名（group name),搜寻每个名字对应的文件tsv,提取其product列，将名称改为其分组名，同一txt的在同一输出文件成行。
import numpy as np
import pandas as pd
group_name=1
tsv_path='/mnt/d/biotools/workshop/ndm_plasmid/output/roary_ndm_assembly_Ecoli/prokka_chro/test/tsv/'
gff_path='/mnt/d/biotools/workshop/ndm_plasmid/output/roary_ndm_assembly_Ecoli/prokka_chro/gff/gff_uniq.txt'

while (group_name < 2):
    with open('./assembly/ndm_group_chro/'+str(group_name)+'.txt','r') as f:
        lines = f.read().splitlines()
        for line in lines:
            tsv=pd.read_table(tsv_path+str(line)+'.tsv')
            gff=pd.read_table(gff_path)
            #print(tsv[tsv['product']=='Metallo-beta-lactamase type 2'])
            ndm_index=tsv[tsv['product']=='Metallo-beta-lactamase type 2'].index.tolist()
            ndm_pi=[ndm - 10 if ndm >10 else 0 for ndm in ndm_index]
            ndm_ai=[ndm + 11 if ndm+11 < tsv.shape[0] else tsv.shape[0] for ndm in ndm_index]
            for pi,ai in zip(ndm_pi,ndm_ai):
                df=tsv.iloc[pi:ai]
                df.reset_index(drop=True,inplace=True)
                #dfdf[df['product']=='Metallo-beta-lactamase type 2'].index.array
                
                ndm_df_index=df[df['product']=='Metallo-beta-lactamase type 2'].index.array[0]
                #print(df.iloc[ndm_df_index,0].split("_")[-2])
                ndm_gff_index=gff[gff["name"].str.contains(df.iloc[ndm_df_index,0].split("_")[-2])].index.array[0]
                if gff.iloc[ndm_gff_index,1] == "-":
                    df=df.iloc[::-1]
                df=df["product"].to_frame()
                df.reset_index(drop=True,inplace=True)
                
                ndm_df_index=df[df['product']=='Metallo-beta-lactamase type 2'].index.array[0]
                add_row=10-ndm_df_index
                for i in range(add_row):
                    df.loc[str(i)]='0'
                df=df.shift(periods=add_row,axis=0,fill_value=0)
                df.reset_index(drop=True,inplace=True)
                df=df.T
                df.rename(index={'product':str(line)},inplace=True)
                df.to_csv('./assembly/ndm_group_chro/'+str(group_name)+'.csv', mode='a', header=False)
    group_name+=1

# circos

## labels

%load ./assembly/A-list.txt
chro：Assembly所有染色体GCA号
chro1：chro去重复
chro_part：GCA每一个独特的基因环境的代表GCA
chro_part1：chro_part去重复
plasmid：Assembly所有质粒的GCA 号
plasmid1：plasmid去重复
plasmid_all：Assembly所有质粒的基因环境（已去重复）
h：在chro1和plasmid1里都有的GCA号
h1：在chro_part1和plasmid1里都有的GCA号
comm_p：提取h在质粒plasmid_all的对应质粒条目
comm_p1：提取h1在质粒plasmid_all的对应质粒条目
comm_c:提取h在染色体chro_all的对应染色体条目
comm_c1：提取h1在染色体chro_all的对应染色体条目

In [ ]:
comm_p=pd.read_table('./assembly/comm_p'+'.txt',header=None).dropna(how='all',axis=1)

In [ ]:
comm_p

In [ ]:
comm_c=pd.read_table('./assembly/comm_c'+'.txt',header=None).dropna(how='all',axis=1)

In [ ]:
comm_c=pd.concat([comm_c[:][0].str.split('.',expand=True)[:][0].to_frame(),comm_c.drop([0],axis=1)],axis=1)

In [ ]:
comm_p=pd.concat([comm_p[:][0].str.split('.',expand=True)[:][0].to_frame(),comm_p.drop([0],axis=1)],axis=1)

In [ ]:
comm_c

In [ ]:
comm_p

In [ ]:
pc_merge=pd.merge(comm_c,comm_p,how='outer',on=0)

In [ ]:
pc_merge=pc_merge.set_index(0)

In [ ]:
pc_merge=pc_merge.drop_duplicates()

In [ ]:
pc_merge.to_csv('./assembly/merge'+'.csv', mode='a', header=False)

## links

### intra GCA

In [ ]:
p_region=pd.read_table('/mnt/c/Users/Yuing/Desktop/circos_test/ndm_pc/p1'+'.txt',header=None).dropna(how='all',axis=0)

In [ ]:
c_region=pd.read_table('/mnt/c/Users/Yuing/Desktop/circos_test/ndm_pc/c1'+'.txt',header=None).dropna(how='all',axis=0)

In [ ]:
p_region

In [ ]:
c_region

In [ ]:
pc_merge=pd.merge(p_region,c_region,how='outer',on=3).dropna(how='any',axis=0)

In [ ]:
pc_merge

In [ ]:
pc_merge.loc[:,'0_x'].str.split('_p',expand=True)[0]

In [ ]:
inner_GCA=pc_merge[pc_merge.loc[:,'0_x'].str.split('_p',expand=True)[0] == pc_merge.loc[:,'0_y'].str.split('_c',expand=True)[0]]

In [ ]:
inner_GCA.iloc[:,[1,2]]

In [ ]:
inner_GCA.to_csv('/mnt/c/Users/Yuing/Desktop/inner_GCA_links'+'.txt',index=False,sep='\t',header=False,float_format = '%.0f')

In [ ]:
inner_GCA.iloc[:,[0,1,2,4,5,6]].to_csv('/mnt/c/Users/Yuing/Desktop/pclinks'+'.txt',index=False,sep='\t',header=False,float_format = '%.0f')

### inter GCA

In [ ]:
p_merge=pd.merge(p_region,p_region,how='outer',on=3)

In [ ]:
p_merge

In [ ]:
p_merge1=p_merge[p_merge.loc[:,'0_x'] != p_merge.loc[:,'0_y']]

In [ ]:
p_merge_list=p_merge1.values.tolist()

In [ ]:
lst1=list(set(_) for _ in p_merge_list)

In [ ]:
l2 = []
l3=[]
for i in p_merge_list:
    if not set(i) in l3:
        l2.append(i)
    l3.append(set(i))
len(l2)

In [ ]:
inter_p = pd.DataFrame(l2,columns=['1','2','3','4','5','6','7'])

In [ ]:
inter_p

In [ ]:
inter_p.to_csv('/mnt/c/Users/Yuing/Desktop/pslinks'+'.txt',index=False,sep='\t',header=False,float_format = '%.0f')

In [ ]:
inter_p.iloc[:,[0,1,2,4,5,6]].to_csv('/mnt/c/Users/Yuing/Desktop/iterp_GCA_links'+'.txt',index=False,sep='\t',header=False,float_format = '%.0f')

## highlights

In [ ]:
h1=p_region[[3]].append(c_region[[3]]).drop_duplicates().reset_index(drop = True)

In [ ]:
h1.shape

In [ ]:
h1

In [ ]:
h2=['fill_color=chr'+str(n+1) for n in range(47)]

In [ ]:
h2=pd.Series(h2)

In [ ]:
h2

In [ ]:
h=pd.concat([h1,h2],axis=1)

In [ ]:
h

dict_h = h.set_index(3).T.to_dict('list')

dict_h

In [ ]:
acc_repl=h.set_index(3)[0].to_dict()

In [ ]:
p_hightlights=p_region.replace(acc_repl)

In [ ]:
c_hightlights=c_region.replace(acc_repl)

In [ ]:
p_hightlights.to_csv('/mnt/c/Users/Yuing/Desktop/p_hightlights'+'.txt',index=False,sep='\t',header=False,float_format = '%.0f')

In [ ]:
c_hightlights.to_csv('/mnt/c/Users/Yuing/Desktop/c_hightlights'+'.txt',index=False,sep='\t',header=False,float_format = '%.0f')

# gene map

In [ ]:
root_dir='/root/jupyter_projects/ndm_plasmid_2021/input/'
dir2='clusters85/'
dir_customs=['1','2','3','4','5','6','7','8','9','10','11','12','13','14']

In [ ]:
ndm_all_processed=pd.read_csv('/root/jupyter_projects/ndm_plasmid_2021/input/'+'ndm_all_processed.csv')

In [ ]:
ndm_species=ndm_all_processed.loc[:][['ACC_NUCCORE','taxon_genus_name','rep2','host_all','Location2']]

In [ ]:
codes, uniques = pd.factorize(ndm_species.taxon_genus_name)

In [ ]:
uniques

#location

sns_colors=sns.color_palette("cubehelix",7).as_hex()[0:8]

#host
sns_colors=sns.color_palette("rocket").as_hex()[1:6]

#rep
eight=sns.color_palette("Paired", 12).as_hex()
eight

seven=sns.color_palette("tab10").as_hex()[1:8]
seven

sns_colors=eight+seven

In [ ]:
#genus
eight=sns.color_palette("Set2", 8).as_hex()
eight

seven=sns.color_palette("Set1", 15).as_hex()[1:9]
seven

sns_colors=eight+seven

In [ ]:
sns_colors

In [ ]:
plt.figure(figsize=(10,3))
fig=sns.color_palette(sns_colors)
fig

In [ ]:
colors_sp=pd.DataFrame(pd.Series(sns_colors,uniques),columns=['color'])

In [ ]:
colors_sp['taxon_genus_name'] = colors_sp.index
colors_sp

In [ ]:
import plotly.express as px
a = 3  # number of rows
b = 6  # number of columns
c = 1  # initialize plot counter
plt.figure(figsize=[30,10])

#for dir_custom,pdf_name in zip(dir_customs,pdf_names):
for dir_custom in dir_customs:

    co=pd.read_csv(root_dir+dir2+'cluster'+dir_custom+'.txt',sep='\t',header=None)
    co.columns=['ACC_NUCCORE']

    co_sp = pd.merge(co, ndm_species, how='left', on=['ACC_NUCCORE'])

    #co_sp['taxon_genus_name'].value_counts()
    unique=co_sp['taxon_genus_name'].value_counts().index
    #code, unique= pd.factorize(co_sp.taxon_genus_name)
    #print(unique)
    
    plt.subplot(a, b, c)
    #patches, labels, pct_texts=
    plt.pie(co_sp['taxon_genus_name'].value_counts(),
       # kind='pie',
        #autopct="%.1f%%",
        colors=colors_sp.loc[unique]['color'].values.tolist(),
        #title="",
        #explode=[0,0.1,0,0,0],                          
        #labels=['']*(len(colors_sp.loc[unique]['color'].values.tolist())),
        #textprops = {'fontsize':12,'color':"black",'family':'sans-serif'},
        #startangle=140,
        #rotatelabels=True,
        #labeldistance=None,
        #pctdistance=0.8,
        #wedgeprops = {'linewidth': 0.01,'edgecolor': 'black'}
        #rotatelabels=True
        #frame=True
    )
    #rotate the texts
    #for label, pct_text in zip(labels, pct_texts):
        #pct_text.set_rotation(label.get_rotation())
    c=c+1
    
    plt.tight_layout()
    plt.legend(colors_sp.loc[unique]['taxon_genus_name'].tolist(),ncol=2,loc='best',prop = {'size':11,'family':'sans-serif'})
    


#plt.savefig('/root/jupyter_projects/ndm_plasmid_2021/output/aroundNDMcluster_s_85.pdf')